# import các thư viện 

In [2]:
import numpy as np 
import pandas as pd 
import random 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import math 
import pandas_datareader as data_reader 

from tqdm import tqdm_notebook , tqdm 
from collections import deque 

# Xây dựng mô hình AI Trade

In [3]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
            
        

# Tiền xử lý tập dữ liệu 

## sigmoid 

In [4]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

## price format function 

In [5]:
def stock_price_format(n):
    if n < 0 :
        return '- $ {:2f}'.format(abs(n))
    else :
        return '$ {:2f}'.format(abs(n))

# Chuẩn bị(nạp) dữ liệu 

In [47]:
def dataset_loader(stock_name):
    dataset = pd.read_excel('/kaggle/input/vnindex/HOSE.xls')
    df2 = df[df['TICKER'] == stock_name]
    df3 = df2.drop(columns=['TICKER'])
    new_df = df3.explode('DATE')
    
    start_date = str(new_df.index[0]).split()[0]
    end_date = str(new_df.index[-1]).split()[0]
    
    new_df.index = df3['DATE']
    
    close = new_df['CLOSE']
    return close  

In [17]:
# Ví dụ mã AAA ở sàn HOSE

df = pd.read_excel('/kaggle/input/vnindex/HOSE.xls')
df2 = df[df['TICKER'] == 'AAA']
df3 = df2.drop(columns=['TICKER'])
df3

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
0,2018-01-03,33.00,33.15,32.50,32.95,144331
1,2018-01-08,32.60,33.60,32.25,33.55,220489
2,2018-01-09,33.60,34.00,33.55,33.95,164117
3,2018-01-10,34.00,34.15,33.50,34.00,118541
4,2018-02-05,28.70,29.00,26.85,26.85,90020
...,...,...,...,...,...,...
407,2019-10-08,15.00,15.10,14.75,14.80,155560
408,2019-10-09,14.85,14.90,14.75,14.80,129026
409,2019-10-10,14.85,15.00,14.80,14.90,127147
410,2019-10-11,14.90,15.05,14.90,14.95,136369


# Tạo trạng thái cho mô hình

In [18]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

# loading a dataset 

In [48]:
stock_name = 'AAA'
data = dataset_loader(stock_name)

data

DATE
2018-01-03    32.95
2018-01-08    33.55
2018-01-09    33.95
2018-01-10    34.00
2018-02-05    26.85
              ...  
2019-10-08    14.80
2019-10-09    14.80
2019-10-10    14.90
2019-10-11    14.95
2019-10-14    15.20
Name: CLOSE, Length: 412, dtype: float64

# Huấn luyện máy học

## Cài đặt các siêu tham số(setting the hyper parameters)

In [49]:
window_size = 10 
episodes = 1000

batch_size = 32
data_samples = len(data) - 1 


## Định nghĩa mô hình giao dịch 

In [50]:
trader = AI_Trader(window_size)

In [51]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# training loop ( # Sẽ mất nhiều thời gian )

In [ ]:
 
 for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stock_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/411 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 33.550000
AI Trader bought:  $ 33.950000
AI Trader sold:  $ 26.850000  Profit: - $ 6.700000
AI Trader sold:  $ 25.000000  Profit: - $ 8.950000
AI Trader bought:  $ 27.700000
AI Trader sold:  $ 28.450000  Profit: $ 0.750000
AI Trader bought:  $ 28.650000
AI Trader sold:  $ 27.900000  Profit: - $ 0.750000
AI Trader bought:  $ 28.200000
AI Trader bought:  $ 27.700000
AI Trader bought:  $ 27.600000
AI Trader bought:  $ 26.900000
AI Trader sold:  $ 27.800000  Profit: - $ 0.400000
AI Trader sold:  $ 27.100000  Profit: - $ 0.600000
AI Trader sold:  $ 26.550000  Profit: - $ 1.050000
AI Trader sold:  $ 27.450000  Profit: $ 0.550000
AI Trader bought:  $ 26.450000
AI Trader sold:  $ 26.550000  Profit: $ 0.100000
AI Trader bought:  $ 26.300000
AI Trader bought:  $ 25.600000
AI Trader sold:  $ 24.800000  Profit: - $ 1.500000


  8%|▊         | 34/411 [00:09<08:01,  1.28s/it]

AI Trader sold:  $ 25.100000  Profit: - $ 0.500000


  9%|▊         | 35/411 [00:12<12:41,  2.03s/it]

...

 69%|██████▉   | 283/411 [16:10<08:05,  3.79s/it]

AI Trader bought:  $ 17.350000


 69%|██████▉   | 284/411 [16:14<08:08,  3.85s/it]

AI Trader sold:  $ 17.700000  Profit: $ 0.350000


 70%|██████▉   | 287/411 [16:26<08:04,  3.91s/it]